# Importar librerías

In [1]:
# Instalar librerías necesarias
!pip install datasets

In [1]:
# Importar librerías requeridas
import os
import re
import numpy as np
import pandas as pd
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sección 1: Preparación de los datos

## 1.1 Cargar el Dataset

El conjunto de datos de AG News es una colección de artículos de noticias, un subconjunto, del corpus de artículos de noticias de AG. Contiene 120.000 muestras de entrenamiento y 7.600 muestras de prueba. Cada muestra se clasifica en una de cuatro clases: Mundo, Deportes, Negocios y Ciencia/Tecnología. Para nuestra tarea de generación de texto, nos centraremos en el contenido del texto, sin tener en cuenta las categorías.


In [3]:
# Cargar el dataset AG News
dataset = load_dataset("ag_news")

# Convertir a un dataframe de Pandas
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
# Mostrar información básica del dataset
print(f"\nForma del conjunto de entrenamiento: {df_train.shape}")
print(f"Forma del conjunto de prueba: {df_test.shape}")
print("\nEjemplo de algunas muestras:")
print(df_train.head())

# Mostrar etiquetas únicas
print("\nEtiquetas únicas")
print(df_train['label'].unique())


Forma del conjunto de entrenamiento: (120000, 2)
Forma del conjunto de prueba: (7600, 2)

Ejemplo de algunas muestras:
                                                text  label
0  Wall St. Bears Claw Back Into the Black (Reute...      2
1  Carlyle Looks Toward Commercial Aerospace (Reu...      2
2  Oil and Economy Cloud Stocks' Outlook (Reuters...      2
3  Iraq Halts Oil Exports from Main Southern Pipe...      2
4  Oil prices soar to all-time record, posing new...      2

Etiquetas únicas
[2 3 1 0]


## 1.2 Preprocesamiento de los datos
El preprocesamiento es un paso crucial en las tareas de procesamiento del lenguaje natural.

Para nuestro modelo de generación de texto, necesitamos limpiar los datos de texto y convertirlos a un formato adecuado para el entrenamiento.

Esto implica eliminar los caracteres innecesarios, convertirlos a minúsculas y tokenizar el texto.

Haremos el preprocesamiento en simultáneo para el modelado a nivel de caracteres y para el modelado a nivel de palabras.

Limpiar el texto de entrenamiento para un modelo de generación de texto puede mejorar la calidad y eficiencia del modelo, pero no siempre es necesario eliminar todos los signos de puntuación. La decisión de limpiar o no el texto depende de varios factores, incluyendo el propósito del modelo y el tipo de texto que se está procesando.

**Ventajas de Mantener los Signos de Puntuación**

1. Los signos de puntuación como comas, puntos y signos de exclamación proporcionan un contexto importante y afectan al significado del texto. Eliminar estos signos puede resultar en la pérdida de información significativa.

2. En muchos casos, los signos de puntuación ayudan a estructurar el texto de manera coherente y fluida, lo cual es crucial para la generación de texto natural y legible.

3. Mantener los signos de puntuación puede ayudar al modelo a aprender a generar texto que suene más natural y esté mejor estructurado, lo que es particularmente útil en tareas de generación de texto como la escritura creativa o la generación de respuestas en chatbots.

**Ventajas de Limpiar los Signos de Puntuación**

1. Al eliminar signos de puntuación, el espacio de entrada se reduce, lo que puede simplificar el modelo y disminuir la carga computacional, especialmente en textos técnicos o de datos donde la puntuación puede no ser crucial.

2. Limpiar el texto puede ayudar a estandarizarlo y reducir la variabilidad, lo que puede ser útil para mejorar la eficiencia del modelo y su capacidad para generalizar.

Lo ideal sería mantener los signos de puntuación para generar un texto más coherente y real, pero esto se traduce diccionarios y modelos muchos más grandes. Necesitando más capacidad de cómputo para predecir la siguiente letra o palabra, capacidad de cómputo de la cual carecemos. Ya que el objetivo es comparar los enfoques del modelado a nivel de caracteres y a nivel de palabras, optamos por eliminar los signos de puntuación y los números.

In [5]:
def preprocess_text(text):
    # Convertir a minúscula
    text = text.lower()
    # Remover los caracteres especiales y los dígitos
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text

# Preprocesar los textos
df_train['processed_text'] = df_train['text'].apply(preprocess_text)
df_test['processed_text'] = df_test['text'].apply(preprocess_text)

# Tokenización a nivel de caracter
char_tokenizer = Tokenizer(char_level=True)
char_tokenizer.fit_on_texts(df_train['processed_text'])

# Tokenización a nivel de palabras
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(df_train['processed_text'])

print("Tamaño del vocabulario (a nivel de caracteres):", len(char_tokenizer.word_index))
print("Tamaño del vocabulario (a nivel de palabras):", len(word_tokenizer.word_index))

# Mostrar ejemplos preprocesados
sample_text = df_train['processed_text'].iloc[0]
print("\nEjemplo preprocesado:")
print(sample_text)

# Mostrar versiones tokenizadas
char_seq = char_tokenizer.texts_to_sequences([sample_text])[0][:50]
word_seq = word_tokenizer.texts_to_sequences([sample_text])[0][:10]

print("\nTokenización a nivel de caracteres (primeros 50 tokens):")
print(char_seq)
print("\nTokenización a nivel de palabras (primeros 10 tokens):")
print(word_seq)

Tamaño del vocabulario (a nivel de caracteres): 27
Tamaño del vocabulario (a nivel de palabras): 91343

Ejemplo preprocesado:
wall st bears claw back into the black reuters reuters  shortsellers wall streets dwindlingband of ultracynics are seeing green again

Tokenización a nivel de caracteres (primeros 50 tokens):
[20, 3, 10, 10, 1, 6, 4, 1, 21, 2, 3, 9, 6, 1, 13, 10, 3, 20, 1, 21, 3, 13, 23, 1, 5, 8, 4, 7, 1, 4, 12, 2, 1, 21, 10, 3, 13, 23, 1, 9, 2, 14, 4, 2, 9, 6, 1, 9, 2, 14]

Tokenización a nivel de palabras (primeros 10 tokens):
[391, 324, 1525, 14260, 99, 54, 1, 812, 23, 23]


# Sección 2: Definición de los modelos

## 2.1 Modelo a Nivel de Caracteres
Nuestro modelo de Redes Neuronales Recurrentes (RNN) a nivel de caracteres estará basado en celdas LSTM (memoria larga a corto plazo). Aprenden a predecir el siguiente caracter de una secuencia, capturando patrones y relaciones entre caracteres. Las LSTM son particularmente útiles porque pueden mantener dependencias a largo plazo en el texto, lo cual es crucial para generar contenido coherente. En este modelo, cada carácter se representa como un vector codificado one-hot y la red aprende a predecir la distribución de probabilidad del siguiente carácter.



In [6]:
# Definir parámetros
char_vocab_size = len(char_tokenizer.word_index) + 1
char_embedding_dim = 50
char_lstm_units = 128
char_sequence_length = 100

# Construir el modelo a nivel de caracteres
char_model = Sequential([
    Embedding(char_vocab_size, char_embedding_dim, input_length=char_sequence_length),
    LSTM(char_lstm_units, return_sequences=True),
    LSTM(char_lstm_units),
    Dense(char_vocab_size, activation='softmax')
])

char_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Mostrar resumen del modelo
print("Resumen del modelo a nivel de caracteres:")
char_model.summary()

Resumen del modelo a nivel de caracteres:


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### **Explicación del modelo**
#### **Capa de Embeddings**
Esta capa convierte el vector one-hot disperso en un embadding, una representación densa y significativa. El modelo ve a cada caracter como un conjunto complejo de características. Estas características capturan relaciones sutiles entre caracters que el modelo puede usar más adelante.

#### Parámetros

**char_vocab_size:** Representa el tamaño del vocabulario de caracteres. Es el número total de caracteres únicos en el conjunto de datos, más uno para el token de inicio.

**char_embedding_dim:** La dimensión de los embeddings de caracteres es 50. Esto significa que cada carácter se representa como un vector de 50 dimensiones.

**char_sequence_length:** La longitud de la secuencia es 100. Esto indica que el modelo procesará secuencias de 100 caracteres a la vez.

#### **Primera capa LSTM:**
Esta capa actúa como un recopilador de contexto. A medida que lee la secuencia de embeddings, realiza un seguimiento de patrones y relaciones importantes en distancias cortas y largas en el texto.

#### Parámetros

**char_lstm_units:** Cada capa LSTM tendrá 128 unidades, lo que define la capacidad de la red para captar patrones y dependencias en los datos.

**return_sequences=True:** Indica que la capa LSTM debe devolver la secuencia completa de salidas para cada paso de tiempo, no solo la última. Es necesario para que la próxima capa LSTM procese toda la secuencia de manera recurrente.

#### **Segunda capa LSTM:**
Esta capa es un analizador más profundo. Toma la información contextual del primer LSTM y la refina aún más.

#### **Capa densa:**
Esta capa final densa es el predictor. Considera todos los caracteres posibles y asigna una probabilidad a cada uno, basándose en todo lo que ha aprendido de la secuencia.

#### Parámetros

**units = char_vocab_size:** El número de unidades en la capa densa es igual al tamaño del vocabulario. Esto asegura que haya una probabilidad de salida para cada carácter en el vocabulario.

**activation='softmax':** La activación softmax convierte las salidas en probabilidades, sumando a 1, para la predicción del siguiente carácter en la secuencia.

#### **Configuración del modelo**

#### Parámetros

**loss='categorical_crossentropy':** Se utiliza la función de pérdida de entropía cruzada categórica porque estamos trabajando con un problema de clasificación múltiple (predicción del siguiente carácter).

**optimizer='adam':** Se utiliza el optimizador Adam, que es el algoritmo de optimización que ajusta individualmente los pesos del modelo basado en el error y con momentum, proporcionando una convergencia rápida y estable.

## 2.2 Modelo a nivel de palabras

El modelo a nivel de palabras opera en un nivel más alto de abstracción en comparación con el modelo a nivel de caracter. En lugar de predecir el siguiente carácter, predice la siguiente palabra en una secuencia. Este enfoque tiene el potencial de capturar las relaciones semánticas entre palabras de manera más efectiva. Cada palabra se representa como un embedding, lo que permite que el modelo aprenda representaciones significativas de palabras en un espacio vectorial continuo.

In [7]:
# Definir parámetros
word_vocab_size = len(word_tokenizer.word_index) + 1
word_embedding_dim = 100
word_lstm_units = 256
word_sequence_length = 20

# Construir el modelo a nivel de palabras
word_model = Sequential([
    Embedding(word_vocab_size, word_embedding_dim, input_length=word_sequence_length),
    LSTM(word_lstm_units, return_sequences=True),
    LSTM(word_lstm_units),
    Dense(word_vocab_size, activation='softmax')
])

word_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Mostrar el resumen del modelo:
print("\nResumen del modelo a nivel de palabras:")
word_model.summary()


Resumen del modelo a nivel de palabras:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 100)           9134400   
                                                                 
 lstm_2 (LSTM)               (None, 20, 256)           365568    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dense_1 (Dense)             (None, 91344)             23475408  
                                                                 
Total params: 33500688 (127.79 MB)
Trainable params: 33500688 (127.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### **Explicación del modelo**

#### **Capa de Embedding:**
Toma cada vector de palabra indexada y la transforma en una representación compacta y signigicativa. En lugar de ver las palabras como unidades aisladas, el modelo ahora percibe cada palabra como un conjunto de características. Estas características capturan las relaciones semánticas entre palabras, lo que permite que el modelo comprenda matices como sinónimos, antónimos o palabras que se usan a menudo en contextos similares.

#### Parámetros

**word_vocab_size:** Es el tamaño del vocabulario de palabras. Es el número total de palabras únicas en el conjunto de datos más uno, para un índice adicional para el token de inicio (o final).

**word_embedding_dim:** La dimensión de los embeddings de palabras es 100. Cada palabra se representa como un vector de 100 elementos.

**word_sequence_length:** La longitud de la secuencia es 20. Esto significa que el modelo procesa secuencias de 20 palabras a la vez.



---



El resto es igual que en el modelo a nivel de caracteres, con las salvedades de lo que representa cada vector de embedding.

En esencia, este modelo consiste en leer una secuencia de palabras, desarrollar una comprensión del significado, el contexto y la estructura del texto y luego utilizar esa comprensión para predecir la siguiente palabra.

La principal diferencia con el modelo a nivel de caracteres es que éste opera en un nivel superior de abstracción. En lugar de aprender sobre las relaciones entre caracteres, se aprende sobre las relaciones entre palabras. Esto le permite capturar patrones lingüísticos más complejos y potencialmente comprender conceptos de nivel superior.

# Sección 3: Entrenamiento de los modelos


## 3.1 Generación de datos
Preparing data for training sequence models involves creating input-output pairs where the input is a sequence of characters or words, and the output is the next character or word. We'll use a sliding window approach to generate these sequences from our preprocessed text data. For character-level models, we'll create overlapping sequences of characters. For word-level models, we'll create sequences of words. This process ensures that our models learn to predict the next token based on the previous context.

To handle large datasets efficiently, we'll use generators to create our training sequences. This approach allows us to generate data on-the-fly during training, significantly reducing memory usage. We'll create separate generators for character-level and word-level models, each yielding batches of input-output pairs as needed during the training process.


La preparación de datos para entrenar modelos secuenciales implica la creación de pares de entrada y salida donde la entrada es una secuencia de caracteres o palabras y la salida es el siguiente carácter o palabra. Vamos a usar el enfoque de ventana deslizante para generar estas secuencias a partir de los datos de los textos preprocesados. Para el modelo a nivel de caracteres, crearemos secuencias de caracteres superpuestas. Para el modelo a nivel de palabras, crearemos secuencias de palabras. Este proceso permite que nuestros modelos aprendan a predecir el siguiente token en función del contexto anterior.

Para manejar grandes conjuntos de datos de manera eficiente, usaremos generadores para crear nuestras secuencias de entrenamiento. Este enfoque nos permite generar datos sobre la marcha durante el entrenamiento, lo que reduce significativamente el uso de memoria y optimizar el uso de recurosos de los cuales carecemos. Crearemos generadores separados para modelos a nivel de caracteres y a nivel de palabras, cada uno de los cuales generará lotes de pares de entrada y salida según sea necesario durante el proceso de capacitación.

In [8]:
def sequence_generator(texts, tokenizer, seq_length, batch_size, char_level=True):
    sequences = tokenizer.texts_to_sequences(texts)
    vocab_size = len(tokenizer.word_index) + 1
    while True:
        for sequence in sequences:
            n_samples = len(sequence) - seq_length
            if n_samples <= 0:
                continue
            for i in range(0, n_samples, batch_size):
                batch_end = min(i + batch_size, n_samples)
                X = np.zeros((batch_end - i, seq_length), dtype=np.int32)
                y = np.zeros((batch_end - i, vocab_size), dtype=np.float32)
                for j in range(i, batch_end):
                    X[j-i] = sequence[j:j+seq_length]
                    y[j-i, sequence[j+seq_length]] = 1
                yield X, y

# Crear generadores
char_seq_gen = sequence_generator(df_train['processed_text'], char_tokenizer, char_sequence_length, 128)
word_seq_gen = sequence_generator(df_train['processed_text'], word_tokenizer, word_sequence_length, 128, char_level=False)

# Calcular pasos por epoch
char_steps = sum(max(0, len(seq) - char_sequence_length) for seq in char_tokenizer.texts_to_sequences(df_train['processed_text'])) // 128
word_steps = sum(max(0, len(seq) - word_sequence_length) for seq in word_tokenizer.texts_to_sequences(df_train['processed_text'])) // 128

print(f"Pasos por epoch para el modelo a nivel de caracteres: {char_steps}")
print(f"Pasos por epoch para el modelo a nivel de palabras: {word_steps}")

# Mostrar un batch de ejemplo
char_sample_X, char_sample_y = next(char_seq_gen)
word_sample_X, word_sample_y = next(word_seq_gen)

print("\nForma del batch de ejemplo del modelo a nivel de caracteres:")
print(f"X: {char_sample_X.shape}, y: {char_sample_y.shape}")
print("\nForma del batch de ejemplo del modelo a nivel de palabras:")
print(f"X: {word_sample_X.shape}, y: {word_sample_y.shape}")

Pasos por epoch para el modelo a nivel de caracteres: 118185
Pasos por epoch para el modelo a nivel de palabras: 15722

Forma del batch de ejemplo del modelo a nivel de caracteres:
X: (33, 100), y: (33, 28)

Forma del batch de ejemplo del modelo a nivel de palabras:
X: (15, 20), y: (15, 91344)


#### **Pasos por Época**

#### Modelo a Nivel de Caracteres

**Pasos por epoch: 118185**

El número de pasos por epoch para el modelo a nivel de caracteres es 118185. Esto indica que, durante cada epoch de entrenamiento, el modelo va a recibir 118185 lotes de datos generados secuencialmente.

#### Modelo a Nivel de Palabras

**Pasos por época: 15722**

El número de pasos por época para el modelo a nivel de palabras es 15722. Esto significa que, en cada época de entrenamiento, el modelo procesará 15722 lotes de datos.

#### Comparación:

El modelo a nivel de caracteres tiene significativamente más pasos por época que el modelo a nivel de palabras. Esto es porque las secuencias de caracteres son más cortas que las secuencias de palabras y el texto se descompone en más secuencias cuando se considera a nivel de caracteres.

#### **Forma del Lote de Ejemplo**

#### Modelo a Nivel de Caracteres

**Forma de X: (33, 100)**

X es la matriz de entrada para el lote actual. El tamaño del lote es 33, lo que significa que hay 33 secuencias de caracteres en este lote. Cada secuencia tiene una longitud de 100 caracteres.

**Forma de y: (33, 28)**

Y es la matriz de salida (etiquetas) para el lote actual. El número de secuencias en el lote es 33, igual al tamaño del lote de X. Cada fila en y es un vector one-hot con 28 posiciones, representando cada posible carácter en el vocabulario. El vocabulario de caracteres incluye 28 caracteres únicos, por lo que cada vector one-hot tiene una dimensión de 28.

#### Modelo a Nivel de Palabras

**Forma de X: (15, 20)**

El tamaño del lote es 15, indicando que hay 15 secuencias de palabras en este lote. Cada secuencia tiene una longitud de 20 palabras.

**Forma de y: (15, 91344)**

El número de secuencias en el lote es igual a 15, igual al tamaño del lote de X.
Cada fila en y es un vector one-hot con 91344 posiciones, representando cada posible palabra en el vocabulario.

## 3.2 Entrenamiendo de los modelos
El entrenamiento de modelos secuenciales implica alimentar las secuencias de entrada a la red, comparar el siguiente token predicho con el siguiente token real y ajustar los pesos del modelo para minimizar esta diferencia. Usaremos el optimizador Adam y la función de pérdida categorical cross-entropy para ambos modelos. Es importante monitorear el proceso de capacitación para evitar un sobreajuste, por lo que usaremos datos de validación y detención anticipada (Early Stopping). Debido al gran conjunto de datos, utilizaremos el entrenamiento por lotes para que el proceso sea más eficiente en cuanto a memoria.

### 3.2.1. Configurar los checkpoints y definir el Early Stopping

In [9]:
# Crear directorios para checkpoints
os.makedirs('checkpoints/char_model', exist_ok=True)
os.makedirs('checkpoints/word_model', exist_ok=True)

# Configurar el Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Configurar el checkpointing para el modelo a nivel de caracteres
char_checkpoint = ModelCheckpoint(
    'checkpoints/char_model/model.{epoch:02d}-{val_loss:.2f}.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)

# Configurar el checkpointing para el modelo a nivel de caracteres
word_checkpoint = ModelCheckpoint(
    'checkpoints/word_model/model.{epoch:02d}-{val_loss:.2f}.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)

### 3.2.2. Cargar los pesos del checkpoint si existiesen para el modelo a nivel de caracteres

In [15]:
# Cargar el mejor modelo a nivel de caracteres
try:
    best_char_model_path = f'checkpoints/char_model/{sorted(os.listdir("checkpoints/char_model"))[-1]}'
    # Verificar si existe un checkpoint, cargar los pesos y mostrar la pérdida inicial
    if os.path.exists(best_char_model_path):
        print("Cargando el checkpoint del modelo a nivel de caracteres...")

        char_model.load_weights(best_char_model_path)
        print("Checkpoint cargado. Evaluando el modelo...")

        evaluation = word_model.evaluate(word_seq_gen, steps=word_steps // 10)
        print(f"Pérdida inicial luego de cargar el checkpoint: {evaluation}")

except IndexError:
    print("No se encontraron elementos en el directorio checkpoints.")



No se encontraron elementos en el directorio checkpoints.


### 3.2.3. Entrenar el modelo a nivel de caracteres

In [ ]:
# Entrenar el modelo a nivel de caracteres
print("Entrenando del modelo a nivel de caracteres...")
char_history = char_model.fit(
    char_seq_gen,
    steps_per_epoch=char_steps,
    epochs=10,
    validation_data=char_seq_gen,
    validation_steps=char_steps // 10,
    callbacks=[early_stopping, char_checkpoint]
)

### 3.2.4. Cargar los pesos del checkpoint si existiesen para el modelo a nivel de palabras

In [16]:
# Cargar el mejor modelo a nivel de palabras
try:
    best_word_model_path = f'checkpoints/word_model/{sorted(os.listdir("checkpoints/word_model"))[-1]}'

    # Verificar si existe un checkpoint, cargar los pesos y mostrar la pérdida inicial
    if os.path.exists(best_word_model_path):
        print("Cargando el checkpoint del modelo a nivel de palabras...")

        word_model.load_weights(best_word_model_path)
        print("Checkpoint cargado. Evaluando el modelo...")

        evaluation = word_model.evaluate(word_seq_gen, steps=word_steps // 10)
        print(f"Pérdida inicial luego de cargar el checkpoint: {evaluation}")
except IndexError:
    print("No se encontraron elementos en el directorio checkpoints.")

No se encontraron elementos en el directorio checkpoints.


### 3.2.5. Entrenar el modelo a nivel de palabras

In [ ]:
# Entrenar el modelo a nivel de palabras
print("Entrenando del modelo a nivel de palabras...")
word_history = word_model.fit(
    word_seq_gen,
    steps_per_epoch=word_steps,
    epochs=10,
    validation_data=word_seq_gen,
    validation_steps=word_steps // 10,
    callbacks=[early_stopping, word_checkpoint]
)

# Sección 4: Generación de texto

## 4.1 Generación a nivel de caracteres

La generación de texto a nivel de caracteres implica predecir un carácter a la vez en función de la secuencia de caracteres anterior. El modelo toma una secuencia semilla como entrada y genera nuevos caracteres uno por uno, incorporando cada vez el carácter recién generado en la secuencia de entrada para la siguiente predicción. Este proceso continúa hasta que alcanzamos la longitud deseada o una condición de parada. La temperatura se utiliza para controlar la aleatoriedad del texto generado.

In [18]:
def generate_text_char(model, tokenizer, seed_text, num_chars=200, temperature=1.0):
    generated_text = seed_text
    for _ in range(num_chars):
        # Tokenizar la secuencia actual
        x = tokenizer.texts_to_sequences([generated_text[-char_sequence_length:]])[0]
        x = pad_sequences([x], maxlen=char_sequence_length, padding='pre', truncating='pre')

        # Predecir el siguiente caracter
        predictions = model.predict(x, verbose=0)[0]

        # Aplicar muestreo de temperatura
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Obtener el caracter siguiente
        next_index = np.random.choice(len(predictions), p=predictions)
        next_char = tokenizer.index_word.get(next_index, ' ')  # Por defecto es un espacio

        # Añadir el caracter generado
        generated_text += next_char

    generated_text += "."
    return generated_text

# Cargar el mejor modelo a nivel de caracteres
best_char_model_path = f'checkpoints/char_model/{sorted(os.listdir("checkpoints/char_model"))[-1]}'
char_model.load_weights(best_char_model_path)

# Gernerar texto usando el modelo a nivel de caracteres
seed_texts = [
    "The future of artificial intelligence",
    "In a world where technology",
    "The most important stocks",
    "The global economy faces",
    "Space exploration "
]

print("Testos generados con el modelo a nivel de caracteres:")
for seed in seed_texts:
    generated_text = generate_text_char(char_model, char_tokenizer, seed)
    print(f"\nSemilla: {seed}")
    print(generated_text)
    print("-" * 50)

Testos generados con el modelo a nivel de caracteres:

Semilla: The future of artificial intelligence
The future of artificial intelligence showed against investy is a trade plidged to vrice new family longs as win russies major its top peace finally handsets for seconds on moneyan said after part of the onedolessy there pass to be decid
--------------------------------------------------

Semilla: In a world where technology
In a world where technology reaccosegry of allows to lesss for his tricken attacks for the microy las of an investigation of mosibility stocks fodd explored prosecutorations out in nithpust declared dips revale on the products 
--------------------------------------------------

Semilla: The most important stocks
The most important stocks are its hesproving next week ago but at offering quot turning in the springfield in recent forced as solutions china weather machines soldweatrum plans to on the savitys israeli said on sunday in the
------------------------------

## 4.2 Generación a nivel de palabras

La generación de texto a nivel de palabra es similar a la generación a nivel de caracteres, pero predice palabras completas en lugar de caracteres individuales. Esto se espera que de como resultado un texto más coherente en secuencias más largas, ya que el modelo ha aprendido patrones y relaciones a nivel de palabras. Sin embargo, puede producir resultados menos diversos en comparación con los modelos a nivel de personaje.

In [20]:
def generate_text_word(model, tokenizer, seed_text, num_words=50, temperature=1.0):
    generated_text = seed_text.split()
    for _ in range(num_words):
        # Tokenizar la secuencia actual
        x = tokenizer.texts_to_sequences([' '.join(generated_text[-word_sequence_length:])])[0]
        x = pad_sequences([x], maxlen=word_sequence_length, padding='pre', truncating='pre')

        # Predecir la siguiente palabra
        predictions = model.predict(x, verbose=0)[0]

        # Aplicar muestreo de temperatura
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Obtener la siguiente palabra
        next_index = np.random.choice(len(predictions), p=predictions)
        next_word = tokenizer.index_word.get(next_index, '<UNK>')  # Usar <UNK> para palabras desconocidas

        # Añadir la palabra generada
        generated_text.append(next_word)

    generated_text += "."

    return ' '.join(generated_text)

# Cargar el mejor modelo a nivel de palabras
best_word_model_path = f'checkpoints/word_model/{sorted(os.listdir("checkpoints/word_model"))[-1]}'
word_model.load_weights(best_word_model_path)

# Generar usando el modelo a nivel de palabras
seed_texts = [
    "The future of artificial intelligence",
    "In a world where technology",
    "The most important stocks",
    "The global economy faces",
    "Space exploration "
]

print("\nTextos generados a nivel de palabras:")
for seed in seed_texts:
    generated_text = generate_text_word(word_model, word_tokenizer, seed)
    print(f"\nSemilla: {seed}")
    print(generated_text)
    print("-" * 50)


Textos generados a nivel de palabras:

Semilla: The future of artificial intelligence
The future of artificial intelligence site a person said on wednesday but it that was paying back out of their bush administration s work in an order to ensure a joint collective bargaining agency an injunction but why may on violent it appears to reduce global warming contained such as cell and gulf of the
--------------------------------------------------

Semilla: In a world where technology
In a world where technology the metres spacecraft is currently in a north korea break to murder people one major rules from scientists and scandals to carry out of the senate along with landslides out of a canadian international delays by murder an american hospital reported the tiny neighbours and toppled marines were in the
--------------------------------------------------

Semilla: The most important stocks
The most important stocks of a milky century in a field pharmacies in the latest wave of other neigh

# Sección 5: Comparación de modelos

## **Estructura del Modelo y Representación de Datos**

### Modelo a Nivel de Caracteres

El modelo a nivel de caracteres procesa el texto como una secuencia de caracteres individuales. El modelo predice el siguiente carácter en la secuencia. La arquitectura utilizada incluye una capa de embeddings para mapear caracteres a vectores que capturen las relaciones entre los caracteres en distintas dimensiones, seguida por dos capas LSTM que capturan la dependencia temporal en la secuencia de caracteres. Finalmente, una capa densa con activación softmax predice la probabilidad del siguiente carácter en el vocabulario.

Tras analizar el resultado de los textos generados, se destaca la flexibilidad, pudiendo generar cualquier secuencia de caracteres, lo que es útil para generar textos creativos y menos dependiente de los textos de entrenamiento. La desventaja es que tiene una visión limitada del contexto global debido a que cada carácter por sí solo contiene poca información semántica.

Este modelo requiere de más pasos de entrenamiento y es menos eficiente en términos de tiempo de cómputo debido a la mayor longitud de las secuencias. Lo que se tradujo en una mayor dificultad de entrenamiento.

### Modelo a Nivel de Palabras
El modelo a nivel de palabras procesa el texto como una secuencia de palabras. Cada palabra es representada como un índice en un vocabulario de palabras, y el modelo predice la siguiente palabra en la secuencia. Este enfoque utiliza una capa de embeddings para convertir las palabras en vectores densos y representativos, seguida de dos capas LSTM para capturar dependencias a largo plazo en las secuencias de palabras. La capa de salida es una softmax que predice la probabilidad de la siguiente palabra en el vocabulario.

Una ventaja, que pudimos observar con respecto al modelo a nivel de caracteres, es que captura mejor las relaciones semánticas y la estructura gramatical del texto, ya que cada palabra aporta más información semántica que un carácter individual. Esto se tradujo en textos más coherentes. Además Maneja secuencias más cortas, lo que puede resultar en una mayor eficiencia computacional y tiempos de entrenamiento más rápidos.

Por otro lado, un vocabulario extenso puede llevar a una mayor dimensionalidad de la salida y a la necesidad de más memoria para almacenar los embeddings y manejar la salida one-hot.

## **Resultados y Calidad del Texto Generado**

### Modelo a Nivel de Caracteres

**Resultados**

El modelo a nivel de caracteres tiende a producir una variedad amplia de secuencias de texto, algunas de las cuales pueden no ser lingüísticamente coherentes o tener errores ortográficos. Sin embargo, este es capaz de generar nuevas palabras o combinaciones de caracteres que no se encuentran en los datos de entrenamiento, lo que puede ser útil para la generación creativa o artística.

Estimamos que sería eficaz en la reproducción de patrones de caracteres locales, si hubiese sido entrenado incluyendo símbolos y dígitos, lo que permitiría generar texto que siguieran las reglas ortográficas y gramaticales en niveles locales.

Por otro lado, la fluidez y coherencia global del texto pueden ser limitadas debido a la falta de contexto semántico amplio.

### Modelo a Nivel de Palabras

**Resultados**

El modelo a nivel de palabras genera textos más coherentes y semánticamente consistentes, con frases y oraciones que tienen sentido global. Así mismo no comete ortográficos y casi no comete  gramaticales al contrario del modelo a nivel de caracteres. Ya que, en primer lugar, predice palabras enteras que ya tienen su ortografía definida y codificada y en segundo, por captar mejor la semántica global.

La generación de texto parece ser más predecible y menos creativa, ya que está limitada por el vocabulario de palabras predefinido y está más influenciada por la gramática del conjunto de entrenamiento.

Este modelo maneja mejor el contexto y la coherencia a largo plazo, produciendo texto que sigue una estructura lógica y gramatical adecuada pero está limitado por las palabras o frases que están presentes en el vocabulario de entrenamiento, lo que restringe la capacidad de generación creativa.

## **Consideraciones para la Aplicación**

El modelo a nivel de caracteres es ideal para tareas que requieren creatividad en la generación de texto, como la poesía o la generación de nombres.También es útil para lenguajes técnicos y códigos que requieren precisión a nivel de caracteres.

Por otro lado, el modelo a nivel de palabras es preferible para la mayoría de las tareas de procesamiento de lenguaje natural, donde la coherencia semántica y la estructura gramatical son críticas. También parece adecuado para la generación de textos técnicos y artículos que requieren precisión y claridad en el lenguaje. Sumado a esto, al poder captar la semántica global de los textos, es mejor para generar textos largos que requieren mantener el contexto y la coherencia a lo largo de múltiples oraciones o párrafos.

## **Otras consideraciones**
Las conclusiones volcadas en el texto anterior fueron en parte conjeturas obtenidas de la observación de los resultados y en parte inferidos a través de la comprensión de la arquitectura de la red y de los procesos de entrenamiento y predicción. Esto se debe a que nos vimos limitados por la capacidad de cómputo prestadas por Colab, Kaggle, entre otros, borrando los runtimes sin permitirnos descargar los útimos checkpoints y perdiendo epochs enteras por este motivo. Además, el tiempo de GPU en estos servicios tiene un tiempo de refresco alto, y nuestro tiempo estuvo limitado. Esto nos impuso restricciones en el tamaño del modelo y en la inclusión de símbolos en el entrenamiento. Por estos motivos el texto generado es de una calidad pobrísima y, por ende, difícil de analizar en profundidad y obtener insights más valiosos.

De haber tenido a disposición un poder de cómputo mayor hubiéramos propuesto las siguientes al modelo aquí presentado:
1. Inclusión de símbolos y dígitos a los diccionarios
2. Aumento de unidades LSTM por capa
3. Añadir más capas densas con dropout para prevenir el overfitting
4. Implementar LSTM bidireccionales
5. Usar un learning rate scheduler como ReduceLROnPlateau
6. Aumentar la longitud de la secuencia
7. Usar arquitecturas más sofisticadas como encoder-decoder, encoder-decoder con atención o incluso transformers.

## **Conclusión**
En conclusión, ambos enfoques tienen sus ventajas y desventajas dependiendo del contexto y la aplicación específica. El modelo a nivel de caracteres es más flexible y creativo, pero puede carecer de coherencia semántica a largo plazo. Por otro lado, el modelo a nivel de palabras es más efectivos para generar textos coherentes y semánticamente ricos, aunque pueden estar limitados por el tamaño y la composición del vocabulario y las muestras de entrenamiento. La elección entre uno y otro dependerá de las necesidades específicas del problema y del tipo de texto que se desea generar.
